# [복습 과제2] 뉴스 본문 수집하기

1. 앞서 실습에서 진행한 데이터프레임을 불러와 학습한 개념을 응용해 각 URL로 접속하여 뉴스 본문을 수집합니다.
2. 제목, (입력) 날짜, 본문, 언론사 정보, URL을 하나의 데이터프레임으로 생성해 추출합니다.
3. 최종적으로 '제목', '날짜', '본문', '언론사', 'url' 5개의 컬럼을 생성한 데이터프레임을 만들어 CSV 파일을 추출합니다. Slack 채널에는 데이터프레임 이미지를 캡처해 댓글로 업로드합니다.

In [80]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks'
!pip install chromedriver-autoinstaller

zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [81]:
# selenium 설치 확인

!python --version

import selenium
print(selenium.__version__)

Python 3.12.2
4.25.0


In [82]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options
import re
from bs4 import BeautifulSoup


In [83]:
#  ChromeDriver를 자동으로 설치하고 경로를 반환합니다.
chrome_path = chromedriver_autoinstaller.install()  

In [ ]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Colab은 새창을 지원하지않기 때문에 창을 띄우지 않는 Headless 모드로 실행해야 합니다.
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

In [84]:
# 셀레니움 라이브러리를 사용하여 Chrome 웹 브라우저를 자동으로 제어하기 위해
# ChromeDriver를 초기화하는 코드
driver = webdriver.Chrome(options=chrome_options)

In [108]:
df = pd.read_csv('프로야구_네이버_뉴스_20240901_20240927.csv')
df

,title,url,date,hits
0,"'김도영 35호포→나성범 동점포→이우성 9회 결승타' 0-5 뒤집은 KIA, 챔피언...",https://m.sports.naver.com/kbaseball/article/0...,20240901,"조회수139,376"
1,"'초비상' 롯데, 팀 최다 홈런 손호영 사구 후 교체...149km 직구 오른손 강...",https://m.sports.naver.com/kbaseball/article/3...,20240901,"조회수131,960"
2,“진짜 말도 안 되는 인생 홈런” 스리런 포함 3안타 5타점 먹방쇼→15-13 승리...,https://m.sports.naver.com/kbaseball/article/1...,20240901,"조회수125,209"
3,"‘예외 없다’ KIA, 2위 삼성 연파하고 6.5게임 차 선두…매직넘버 12",https://m.sports.naver.com/kbaseball/article/1...,20240901,"조회수122,346"
4,"한화이글스, 올해도 가을 야구는 멀어지나",https://m.sports.naver.com/kbaseball/article/6...,20240901,"조회수114,159"
...,...,...,...,...
515,"[공식] 배찬승 4억-구단 역대 2위→원태인보다 많다, 삼성 2025 신인 계약…심...",https://m.sports.naver.com/kbaseball/article/4...,20240927,"조회수62,457"
516,"김도영, 와이스→반즈→하트? 왜 외인 에이스만 계속 만나나…'토종 최초 40-40'...",https://m.sports.naver.com/kbaseball/article/3...,20240927,"조회수61,783"
517,"'MVP 예약' KIA 김도영→오늘(27일) 홈런 기대감 높은 이유, 드디어 다시 ...",https://m.sports.naver.com/kbaseball/article/1...,20240927,"조회수59,023"
518,"'미친 폭염' 혼난 KBO, 2025시즌 7·8월 일요일·공휴일 오후 6시 개시 변...",https://m.sports.naver.com/kbaseball/article/3...,20240927,"조회수58,587"


In [24]:
# 뉴스 본문
from tqdm.auto import tqdm


news_contents = []
for url in tqdm(df['url']):
    driver.get(url)
    
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    article_content = soup.find('div', class_='_article_content')
    
    # 'em' 태그의 class='img_desc'의 텍스트 제거 
    for em_tag in article_content.find_all('em', class_='img_desc'):
        em_tag.decompose()
        
    # <br><br> 태그는 두 칸 띄우기
    for br in article_content.find_all('br'):
        br.replace_with("\n\n")

    
    # article_content는 BeautifulSoup의 객체로, 특정 HTML 요소 나타낸다.
    # 해당 요소에서 텍스트 추출! 
    # get_text()해당 요소, 자식 요소에 포함된 모든 텍스트를 문자열 형태로 추출
    article_text = article_content.get_text(strip=True)
    news_contents.append(article_text)

  0%|          | 0/540 [00:00<?, ?it/s]

In [86]:
news_contents[0]

'[대구=스포츠조선 김영록 기자] \'2위 사냥꾼\' KIA 타이거즈의 명성이 또한번 발휘됐다. 2위 삼성 라이온즈를 저멀리 차내며 거침없는 1위 질주를 이어갔다.KIA는 1일 대구 삼성라이온즈파크에서 열린 삼성전에서 9회초 터진 이우성의 결승타로 6대5, 기적같은 역전승을 거뒀다.이날 승리로 KIA는 75승째(49패2무)를 기록, 2위 삼성을 6경기반 차이로 멀리 떨쳐내며 굳건한 선두를 유지했다. 2017년 이후 첫 정규시즌-한국시리즈 통합 우승이 눈앞으로 다가왔다. 올시즌 NC 다이노스를 시작으로 LG 트윈스, 두산 베어스 등 숱한 팀들이 2위까지 추격했다가 KIA와의 시리즈에서 무너졌다. 삼성도 예외가 되지 못했다.삼성은 56패째(69승2무)를 기록했다. 박병호가 연타석포를 가동하며 2경기 3홈런을 쏘아올린 이번 시리즈다. 하지만 2경기 연속 \'돌부처\' 오승환이 고비 때마다 무너지며 올시즌 가장 중요한 순간 2연패를 막지 못했다.이날 KIA는 박찬호(유격수) 소크라테스(중견수) 김도영(3루) 최형우(좌익수) 나성범(지명타자) 김선빈(2루) 이우성(1루) 김태군(포수) 최원준(우익수)으로 맞섰다. 선발은 에릭 스타우트.삼성은 김지찬(중견수) 김헌곤(우익수) 구자욱(좌익수) 디아즈(지명타자) 박병호(1루) 강민호(포수) 전병우(3루) 안주형(2루) 양도근(유격수) 라인업으로 경기에 임했다. 선발은 원태인.제임스 네일의 대체 외국인 선수로 합류한 KIA 스타우트는 이날이 KBO리그 데뷔전이었다. 선발의 무게감은 일단 원태인 쪽으로 쏠렸지만, 스타우트의 기량에 따라 향후 정규시즌 판도도 파도칠 가능성이 높다.삼성은 류지혁-김영웅에 이어 전날 수비 도중 이재현마저 부상으로 빠진 공백이 크게 느껴진다. 유격수로 나선 양도근은 장안고-강릉영동대를 거쳐 올해 삼성에 육성선수로 입단한 신인 내야수다. 이날 경기가 1군 데뷔전이었다.KIA는 2회초 최형우 나성범의 연속 안타에 이은 김선빈의 희생번트로 먼저 득점 찬스를 잡았다. 하지만 김태군이 우익수 쪽 짧은 뜬공에 그

In [103]:
url = 'https://m.sports.naver.com/kbaseball/article/109/0005147668'
driver.get(url)
    
time.sleep(2)
    
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
    
    # soup.find('img')가 반환하는 객체는 리스트도 아니고 데이터프레임도 아님
    # TAG객체로, HTML 또는 XML의 단일 요소를 나타낸다. 
    # []를 통해 속성에 접근할 수 잇다. 
    # <img> 태그를 나타내는 Tag객체를 반환 후 alt속성의 값을 반환한다
    
news_press = []
    
try:
    news_section = soup.find('h3')
    
        # 이미지 태그
    img = news_section.find('img')
        # 이미지 태그의 alt 속성 값
    news_agent = img['alt']
except Exception as e:            # 예외 발생 시
    news_agent = soup.find('em', class_="NewsEndMain_highlight__HWvAi").get_text()

finally:
    news_press.append(news_agent)
news_press

    
#news_l = soup.find('h3')
#news_2 = news_l.find('alt').get_text()
#news_2

#img = news_l.find('img')  # <img> 태그 찾기
#alt_text = img['alt']   # alt 속성 값 가져오기
#alt_text
    
#news_section = soup.find(class_="NewsEndMain_image_media__rTvT1")#
#news_section

['OSEN']

In [104]:
# 언론사 이름
cnt = 0
news_press = []
for url in tqdm(df['url']):
    driver.get(url)
    
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        news_section = soup.find('h3')
    
        # 이미지 태그
        img = news_section.find('img')
        # 이미지 태그의 alt 속성 값
        news_agent = img['alt']
    except Exception as e:# 예외 발생 시
        news_agent = soup.find('em', class_="NewsEndMain_highlight__HWvAi").get_text()

    finally:
        news_press.append(news_agent)

  0%|          | 0/520 [00:00<?, ?it/s]

In [126]:
news_press[5]

'스포티비뉴스'

In [128]:
# 조회수 열 삭제
df = df.drop('hits', axis = 1)

In [129]:
df['press'] = news_press
# df['content'] = news_contents
len(df['press'])

520

In [130]:
news_contents = news_contents[:520]
len(news_contents)

520

In [131]:
df['content'] = news_contents

In [133]:
df = df[['title', 'date', 'content', 'press', 'url']]

In [134]:
df

,title,date,content,press,url
0,"'김도영 35호포→나성범 동점포→이우성 9회 결승타' 0-5 뒤집은 KIA, 챔피언...",20240901,[대구=스포츠조선 김영록 기자] '2위 사냥꾼' KIA 타이거즈의 명성이 또한번 발...,스포츠조선,https://m.sports.naver.com/kbaseball/article/0...
1,"'초비상' 롯데, 팀 최다 홈런 손호영 사구 후 교체...149km 직구 오른손 강...",20240901,"(엑스포츠뉴스 잠실, 김지수 기자) 롯데 자이언츠 내야수 손호영이 9월 첫 경기 첫...",엑스포츠뉴스,https://m.sports.naver.com/kbaseball/article/3...
2,“진짜 말도 안 되는 인생 홈런” 스리런 포함 3안타 5타점 먹방쇼→15-13 승리...,20240901,"[OSEN=대구, 손찬익 기자] “에너지가 넘칠 수밖에 없는 경기다. 오늘 경기의 ...",OSEN,https://m.sports.naver.com/kbaseball/article/1...
3,"‘예외 없다’ KIA, 2위 삼성 연파하고 6.5게임 차 선두…매직넘버 12",20240901,[데일리안 = 김태훈 기자] 새 외국인 투수 에릭 스타우트는 기대에 미치지 못했지만...,데일리안,https://m.sports.naver.com/kbaseball/article/1...
4,"한화이글스, 올해도 가을 야구는 멀어지나",20240901,5강 싸움 5위 kt에 일격 당해 2.5게임차로 벌어져잔여 22경기 중 1-3위 상...,대전일보,https://m.sports.naver.com/kbaseball/article/6...
...,...,...,...,...,...
515,"[공식] 배찬승 4억-구단 역대 2위→원태인보다 많다, 삼성 2025 신인 계약…심...",20240927,"[OSEN=창원, 조형래 기자] “마지막 인사도 못 드렸는데…”프로야구 NC 다이노...",스포츠서울,https://m.sports.naver.com/kbaseball/article/4...
516,"김도영, 와이스→반즈→하트? 왜 외인 에이스만 계속 만나나…'토종 최초 40-40'...",20240927,[스타뉴스 | 광주=김동윤 기자]롯데 자이언츠 외국인 타자 빅터 레이예스(30)가 ...,엑스포츠뉴스,https://m.sports.naver.com/kbaseball/article/3...
517,"'MVP 예약' KIA 김도영→오늘(27일) 홈런 기대감 높은 이유, 드디어 다시 ...",20240927,[스포티비뉴스=최민우 기자] 부상 공백이 느껴지지 않는다. 오랜 시간 경기에 뛰지 ...,스타뉴스,https://m.sports.naver.com/kbaseball/article/1...
518,"'미친 폭염' 혼난 KBO, 2025시즌 7·8월 일요일·공휴일 오후 6시 개시 변...",20240927,"[OSEN=잠실, 한용섭 기자] 프로야구 한화 이글스는 시즌 막판까지 5강 희망을 ...",엑스포츠뉴스,https://m.sports.naver.com/kbaseball/article/3...


In [136]:
import os

path = '/Users/ranny/GDSC EWHA/'
file_name = '프로야구_네이버_뉴스_본문.csv'

df.to_csv(os.path.join(path, file_name), index=False)